In [99]:
import pandas as pd
import math
import numpy as np

In [103]:
scores = pd.read_csv("data/merged.csv", index_col=0)
#scores["streamed_at"] = pd.to_datetime(scores["streamed_at"])
scores.loc[0]

streamed_at                                     2023-10-09 11:00:21+00:00
track_name                                                   All Hallows'
album_name                                                  The Number 23
album_artist                                                    CASISDEAD
track_id                                           6ULlKqfE82gMmXr7XEJVrE
ms_played                                                           740.0
ip_addr                                                     80.233.60.194
reason_start                                                     clickrow
reason_end                                                        endplay
shuffle                                                             False
skipped                                                              True
full_track_name                                  CASISDEAD - All Hallows'
conn_country                                                           IE
added_at                              

In [71]:
scores.reason_start.value_counts()

reason_start
fwdbtn               16348
clickrow              9083
trackdone             7890
backbtn                667
playbtn                237
trackerror             187
remote                  65
appload                 63
unknown                  3
switched-to-audio        1
Name: count, dtype: int64

In [72]:
scores.reason_end.value_counts()

reason_end
fwdbtn                          16343
endplay                          9186
trackdone                        7786
backbtn                           662
unexpected-exit-while-paused      369
remote                            109
trackerror                         62
logout                             23
unexpected-exit                     4
Name: count, dtype: int64

# Evaulate the fraction played

In [59]:
temp_df = scores[scores.fraction_played > 1]
pd.to_datetime(temp_df.streamed_at)

85      2023-10-11 15:48:41+00:00
190     2023-10-12 18:13:17+00:00
353     2023-10-15 14:51:48+00:00
421     2023-10-19 05:51:16+00:00
430     2023-10-19 06:13:28+00:00
                   ...           
33948   2025-05-07 10:19:03+00:00
34001   2025-05-08 09:01:37+00:00
34112   2025-05-09 06:37:30+00:00
34120   2025-05-09 08:02:49+00:00
34182   2025-05-09 13:01:11+00:00
Name: streamed_at, Length: 403, dtype: datetime64[ns, UTC]

In [60]:
scores.loc[0]

streamed_at                                     2023-10-09 11:00:21+00:00
track_name                                                   All Hallows'
album_name                                                  The Number 23
album_artist                                                    CASISDEAD
track_id                                           6ULlKqfE82gMmXr7XEJVrE
ms_played                                                             740
ip_addr                                                     80.233.60.194
reason_start                                                     clickrow
reason_end                                                        endplay
shuffle                                                             False
skipped                                                              True
full_track_name                                  CASISDEAD - All Hallows'
conn_country                                                           IE
added_at                              

In [96]:
def explode_rows(df: pd.DataFrame) -> pd.DataFrame:
    """
    Function takes a dataframe of spotify listesns and explodes any streams which encapsulate mulitple listens.
    I.e. Any songs that have been listened to more than once will be converted into multiple streams/rows

    Args:
        df: Dataframe of tracks

    Rerturns:
        A DataFrame of tracks where long streams are broken down 
    """

    single_listens = df[df.fraction_played <= 1].copy()
    multi_listens = df[df.fraction_played > 1].copy()

    # For each row we want to take the veiling of the current fraction to get the total number of listens
    multi_listens["stream_count"] = multi_listens.fraction_played.apply(math.ceil)
    # Duplicate 
    new_rows = []
    for _, row in multi_listens.iterrows():
        for stream in range(row.stream_count):
            new_row = row.copy()
            # If the last stream
            if stream == row.stream_count - 1:
                # Played for remainder
                new_row["fraction_played"] = row.fraction_played % 1
                new_row["ms_played"] = new_row["track_duration_ms"] * (row.fraction_played % 1)
                new_row["reason_start"] = "trackdone"
            else:
                new_row["fraction_played"] = 1.0
                new_row["ms_played"] = new_row["track_duration_ms"]
                # Change start resaon if not first stream
                if stream > 0:
                    new_row["reason_start"] = "trackdone"
                new_row["reason_end"] = "trackdone"
            # Start time is now based on what listen we are in the sequence
            new_row["streamed_at"] = new_row["streamed_at"] + pd.Timedelta(milliseconds=stream * new_row["track_duration_ms"])
            new_rows.append(new_row)
    

    # Sense Check
    assert len(new_rows) == multi_listens["stream_count"].sum()
    
    return pd.concat([single_listens, pd.DataFrame(new_rows)[single_listens.columns]])
    

In [97]:
#Fix the start and end reasons 

In [98]:
explode_rows(df=scores)

,streamed_at,track_name,album_name,album_artist,track_id,ms_played,ip_addr,reason_start,reason_end,shuffle,...,track_duration_ms,track_popularity,playlists,album_artwork_url,fraction_played,start_score,end_score,is_saved,enjoyment_score,enjoyment_score_norm
0,2023-10-09 11:00:21+00:00,All Hallows',The Number 23,CASISDEAD,6ULlKqfE82gMmXr7XEJVrE,740.0,80.233.60.194,clickrow,endplay,False,...,166535,48,"['Liked Songs', 'Next Era']",https://i.scdn.co/image/ab67616d0000b273d8de01...,0.004444,1.0,0.0,True,1.004444,0.500373
1,2023-10-09 11:02:54+00:00,A Cat's Demise,"The Girl, the Cat and the Tree",LAUSSE THE CAT,0NB6I1DtmAaJFnvYIvS4j9,154749.0,80.233.60.194,clickrow,trackdone,False,...,154749,35,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b2739ce942...,1.000000,1.0,1.0,True,4.000000,1.000000
2,2023-10-09 11:07:32+00:00,Life in the Balance,Nature's Greatest Mystery,The Four Owls,1345xOev1kyaeqRdeTxiLi,275826.0,80.233.60.194,trackdone,trackdone,False,...,275826,40,[],https://i.scdn.co/image/ab67616d0000b27311eb6f...,1.000000,0.0,1.0,False,2.000000,0.666421
3,2023-10-09 11:26:45+00:00,The Essence,Wamp 2 Dem,Giggs,2j36iWSMTg412fdTQx4WdI,215288.0,80.233.60.194,trackdone,trackdone,False,...,228494,57,['UK Chill'],https://i.scdn.co/image/ab67616d0000b2730aa820...,0.942204,0.0,1.0,True,2.942204,0.823571
4,2023-10-09 11:32:19+00:00,Castles,Blacklisted,Skepta,2pDZ9MQ3n6nMWUzsuTyzYk,206468.0,78.16.229.70,trackdone,trackdone,False,...,206468,54,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b2739f488d...,1.000000,0.0,1.0,True,3.000000,0.833211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34112,2025-05-09 06:41:57.235000+00:00,"Rocky Took a Lover (Le Poisson Rouge, New York)",Field Recordings,Bell X1,3Z7ohV3KU6ZdnIccz7gPdW,243544.0,80.233.39.73,trackdone,trackdone,True,...,267235,45,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b27341a811...,0.911348,1.0,1.0,True,4.911348,1.000000
34120,2025-05-09 08:02:49+00:00,"Rocky Took a Lover (Le Poisson Rouge, New York)",Field Recordings,Bell X1,3Z7ohV3KU6ZdnIccz7gPdW,267235.0,80.233.39.73,clickrow,trackdone,True,...,267235,45,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b27341a811...,1.000000,1.0,-0.5,True,2.360471,0.726544
34120,2025-05-09 08:07:16.235000+00:00,"Rocky Took a Lover (Le Poisson Rouge, New York)",Field Recordings,Bell X1,3Z7ohV3KU6ZdnIccz7gPdW,229948.0,80.233.39.73,trackdone,fwdbtn,True,...,267235,45,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b27341a811...,0.860471,1.0,-0.5,True,2.360471,0.726544
34182,2025-05-09 13:01:11+00:00,Bad Words,Bad Words,Wombat,7bcxU4eDynUXnNhPq7ZEC8,219782.0,80.233.39.73,clickrow,trackdone,True,...,219782,37,['Liked Songs'],https://i.scdn.co/image/ab67616d0000b27323f952...,1.000000,1.0,1.0,True,4.938111,1.000000


85       -6092055361383070246
190       7601050317321371326
353       -570654183503193240
421       6884028448174206439
430      -1187904199137524559
                 ...         
33948     -774234859391516911
34001    -5821101889851740057
34112     4236357851782787229
34120     2196077691287099337
34182     7376531817032888698
Length: 403, dtype: object